<a href="https://colab.research.google.com/github/louistrue/learn-ifc-bfh25-B/blob/main/BFH_25_IFC_Dashboard_Starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/louistrue/learn-ifc-bfh25-B/blob/main/BFH-25-IFC-Dashboard-Starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# IFC Dashboard Starter Template

A minimal template to get started building IFC data dashboards.

**What this does:**
- Load an IFC file from GitHub
- Extract element types and building storeys
- Create a simple 2-tab dashboard


## Step 1: Install packages


In [2]:
%pip install -q ifcopenshell pandas plotly dash


## Step 2: Load IFC file from GitHub


In [3]:
import urllib.request
import ifcopenshell

# Download IFC file from GitHub
github_url = 'https://raw.githubusercontent.com/louistrue/learn-ifc-bfh25-B/main/FireRating/04_CT.ifc'
local_filename = '04_CT.ifc'

print("📥 Downloading IFC file...")
urllib.request.urlretrieve(github_url, local_filename)
print("✅ Downloaded successfully!")

# Open IFC file
model = ifcopenshell.open(local_filename)
print(f"\n🏗️ Loaded: {model.schema}")
print(f"📊 Total elements: {len(model.by_type('IfcElement'))}")


📥 Downloading IFC file...
✅ Downloaded successfully!

🏗️ Loaded: IFC2X3
📊 Total elements: 1943


## Step 3: Parse IFC data

Extract elements and their building storeys.


In [ ]:
import ifcopenshell.util.element as uel
import pandas as pd

element_data = []

for element in model.by_type('IfcElement'):
    element_id = element.GlobalId
    element_type = element.is_a()
    element_name = getattr(element, 'Name', None) or 'Unnamed'

    # Property Sets extrahieren
    psets = uel.get_psets(element)

    # Nach Fire Rating Property suchen
    fire_rating = None
    for pset_name, properties in psets.items():
        # Suche nach verschiedenen Property-Namen für Fire Rating
        possible_names = ['FireRating', 'fire_rating', 'FireRatingName', 'FireResistance']
        for prop_name, prop_value in properties.items():
            if prop_name in possible_names:
                fire_rating = prop_value
                break
        if fire_rating:
            break

    # Finde Geschoss
    storey = 'Not assigned'
    for rel in model.by_type('IfcRelContainedInSpatialStructure'):
        if element in rel.RelatedElements:
            if rel.RelatingStructure and rel.RelatingStructure.is_a('IfcBuildingStorey'):
                storey = rel.RelatingStructure.LongName or rel.RelatingStructure.Name
                break

    element_data.append({
        'ElementId': element_id,
        'ElementType': element_type,
        'ElementName': element_name,
        'FireRating': fire_rating or 'Nicht klassifiziert',
        'Storey': storey
    })

df = pd.DataFrame(element_data)

## Step 4: Create aggregations for visualization


In [ ]:
# Fire Rating Verteilung
fire_rating_counts = (
    df.groupby('FireRating')['ElementId']
    .nunique()
    .reset_index(name='Count')
    .sort_values('Count', ascending=False)
)

# Fire Rating nach Elementtyp
fire_rating_by_type = (
    df.groupby(['FireRating', 'ElementType'])['ElementId']
    .nunique()
    .reset_index(name='Count')
    .sort_values('Count', ascending=False)
)

# Fire Rating nach Geschoss
fire_rating_by_storey = (
    df.groupby(['FireRating', 'Storey'])['ElementId']
    .nunique()
    .reset_index(name='Count')
    .sort_values('Count', ascending=False)
)


## Step 5: Build a simple dashboard


In [ ]:
from dash import Dash, dcc, html
import plotly.express as px

app = Dash(__name__)

# Diagramme erstellen
fig_fire_overview = px.bar(
    fire_rating_counts,
    x='FireRating',
    y='Count',
    title='Fire Rating Verteilung',
    text_auto='.0f'
)

fig_fire_by_type = px.bar(
    fire_rating_by_type,
    x='ElementType',
    y='Count',
    color='FireRating',
    title='Fire Rating nach Elementtyp'
)

fig_fire_by_storey = px.bar(
    fire_rating_by_storey,
    x='Storey',
    y='Count',
    color='FireRating',
    title='Fire Rating nach Geschoss'
)

# Heatmap
fig_heatmap = px.density_heatmap(
    df[df['FireRating'] != 'Nicht klassifiziert'],
    x='ElementType',
    y='FireRating',
    z='Count',
    title='Fire Rating vs Elementtyp'
)

app.layout = html.Div([
    html.H1('Fire Rating Dashboard'),
    html.P('Brandschutzklassifikation der Gebäudeelemente'),

    dcc.Tabs([
        dcc.Tab(label='Übersicht', children=[
            html.H3('Gesamtverteilung'),
            dcc.Graph(figure=fig_fire_overview)
        ]),
        dcc.Tab(label='Nach Elementtyp', children=[
            html.H3('Fire Rating nach Bauteiltyp'),
            dcc.Graph(figure=fig_fire_by_type)
        ]),
        dcc.Tab(label='Nach Geschoss', children=[
            html.H3('Fire Rating pro Geschoss'),
            dcc.Graph(figure=fig_fire_by_storey)
        ]),
        dcc.Tab(label='Heatmap', children=[
            html.H3('Fire Rating Heatmap'),
            dcc.Graph(figure=fig_heatmap)
        ])
    ])
])

app.run(jupyter_mode='inline', height=900, port=8050)


## Next Steps

**Extend this template:**
- Add more tabs for different analyses
- Include materials data
- Add filters with dropdowns
- Try different chart types

**Try other files:**
- Change the `github_url` to load different IFC files
- Upload your own IFC files using file upload widgets

**Learn more:**
- [IFCOpenShell Documentation](https://docs.ifcopenshell.org/)
- [Plotly Dash Documentation](https://dash.plotly.com/)
- [Full example: BFH-25-Tabbed-Dashboard.ipynb](https://github.com/louistrue/learn-ifc-bfh25-B/blob/main/BFH-25-Tabbed-Dashboard.ipynb)
